In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cvxopt
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [ ]:
#cvxopt_solvers.options['show_progress'] = False
#solvers.options['show_progress'] = True

# Importations dataframe

In [ ]:
Xtr0_mat100 = pd.read_csv("/Users/dorian/Downloads/machine-learning-with-kernel-methods-2021/Xtr0_mat100.csv"
                          , header=None, sep = ' ')

Xtr1_mat100 = pd.read_csv("/Users/dorian/Downloads/machine-learning-with-kernel-methods-2021/Xtr1_mat100.csv"
                          , header=None, sep = ' ')

Xtr2_mat100 = pd.read_csv("/Users/dorian/Downloads/machine-learning-with-kernel-methods-2021/Xtr2_mat100.csv"
                          , header=None, sep = ' ')

Ytr0 = pd.read_csv("/Users/dorian/Downloads/machine-learning-with-kernel-methods-2021/Ytr0.csv")

Ytr1 = pd.read_csv("/Users/dorian/Downloads/machine-learning-with-kernel-methods-2021/Ytr1.csv")

Ytr2 = pd.read_csv("/Users/dorian/Downloads/machine-learning-with-kernel-methods-2021/Ytr2.csv")

Xte0_mat100 = pd.read_csv("/Users/dorian/Downloads/machine-learning-with-kernel-methods-2021/Xte0_mat100.csv", 
                         header=None, sep = ' ')

Xte1_mat100 = pd.read_csv("/Users/dorian/Downloads/machine-learning-with-kernel-methods-2021/Xte1_mat100.csv", 
                         header=None, sep = ' ')

Xte2_mat100 = pd.read_csv("/Users/dorian/Downloads/machine-learning-with-kernel-methods-2021/Xte2_mat100.csv", 
                         header=None, sep = ' ')

### Conversion en nd.array
Xtr0_mat100 = Xtr0_mat100.to_numpy()
Xtr1_mat100 = Xtr1_mat100.to_numpy()
Xtr2_mat100 = Xtr2_mat100.to_numpy()

Xte0_mat100 = Xte0_mat100.to_numpy()
Xte1_mat100 = Xte1_mat100.to_numpy()
Xte2_mat100 = Xte2_mat100.to_numpy()

Y0 = Ytr0['Bound'].to_numpy()
Y1 = Ytr1['Bound'].to_numpy()
Y2 = Ytr2['Bound'].to_numpy()

X0 =np.concatenate((Xtr0_mat100, Xte0_mat100), axis=0)
X1 =np.concatenate((Xtr1_mat100, Xte1_mat100), axis=0)
X2 =np.concatenate((Xtr2_mat100, Xte2_mat100), axis=0)

In [86]:
chemin = "/users/dorian/Documents/M2_MASH/Kernel Methods/Data/"
training = pd.read_csv(chemin + "Xtr0.csv")
training2 = pd.read_csv(chemin + "Xtr1.csv")
training3 = pd.read_csv(chemin + "Xtr2.csv")

label = pd.read_csv(chemin + "Ytr0.csv")
label2 = pd.read_csv(chemin + "Ytr1.csv")
label3 = pd.read_csv(chemin + "Ytr2.csv")

test = pd.read_csv(chemin + "Xte0.csv")
test2 = pd.read_csv(chemin + "Xte1.csv")
test3 = pd.read_csv(chemin + "Xte2.csv")

In [96]:
test.head()

,Id,seq
0,0,AAGGCCGAGCCCGGCGCGGACGCAGGCGGCTCCGGGCGGGCTCAGC...
1,1,TCTGGGCTCTTAATGTAAAGGTTGCCACTGATGCTGTGTCACCAGC...
2,2,GCCCGCACCGCTGGGCTATTTTTAGCGTCCACTAAACTTAGCCGAC...
3,3,GAGGCGCTGGCAATGGACTAGGAAGCTCGGCTGCCGCTGCTACTGC...
4,4,GGCGAGACTCCATCTCTACAGAAAATTTTTTTAAAAATTAGCTGGA...


In [14]:
def transcription(ADN):
    Tr =[]
    for g in ADN:
        if g =="A":
            Tr.append(1)
            Tr.append(0)
            Tr.append(0)
            Tr.append(0)
        elif g == "C":
            Tr.append(0)
            Tr.append(1)
            Tr.append(0)
            Tr.append(0)
        elif g =="G":
            Tr.append(0)
            Tr.append(0)
            Tr.append(1)
            Tr.append(0)
        else:
            Tr.append(0)
            Tr.append(0)
            Tr.append(0)
            Tr.append(1)
    return Tr
            

In [482]:
def Ridge(K,y,lbd):
    n = y.shape[0]
    alpha = np.linalg.inv(K+n*lbd*np.eye(n)).dot(y)
    
    return(K.dot(alpha))

def get_alpha(K,y,lbd):
    n = y.shape[0]
    alpha = np.linalg.inv(K+n*lbd*np.eye(n)).dot(y)
    
    return(alpha)

def gaussK(x,y,alpha):
    return np.exp(-alpha*np.linalg.norm(x-y,2)**2)

n,p = training.shape

In [645]:
y = label.iloc[:,1].to_numpy()
y2 = label2.iloc[:,1].to_numpy()
y3 = label3.iloc[:,1].to_numpy()

Xbis = np.zeros((n,4*101))   #Decalage des indices, Xbis pour X1 en vrai
Xter = np.zeros((n,4*101))
X3 = np.zeros((n,4*101))

for i in range(n):
    Xbis[i,] = transcription(training.iloc[i,1])
    Xter[i,] = transcription(training2.iloc[i,1])
    X3[i,] = transcription(training3.iloc[i,1])
    

In [568]:
n_bis = test.shape[0]
n_bis

Xtest1 = np.zeros((n_bis,4*101))
Xtest2 = np.zeros((n_bis,4*101))
Xtest3 = np.zeros((n_bis,4*101))

for i in range(n_bis):
    Xtest1[i,] = transcription(test.iloc[i,1])
    Xtest2[i,] = transcription(test2.iloc[i,1])
    Xtest3[i,] = transcription(test3.iloc[i,1])

In [647]:
y[y == 0] = -1
y2[y2 == 0] = -1
y3[y3 == 0] = -1

## Logistic regression

In [249]:
def solveRR(y, X, lam):
    n,p = X.shape
    assert (len(y) == n)
    
    A = X.T @ X
    # Adjust diagonal due to Ridge
    A[np.diag_indices_from(A)] += lam*n
    b = X.T.dot(y)
    
    # Find solution to the linear system Ax = b
    beta = np.linalg.solve(A,b)
    return (beta)

In [250]:
def solveWRR(y, X, w, lam):
    y1 = np.sqrt(w)*y
    X1 = (np.sqrt(w)*X.T).T
    
    beta = solveRR(y1, X1, lam)
    return (beta)

In [251]:
def solveLRR(y, X, lam):
    # Parameters
    L = 100
    eps = 1e-3
    sigmoid = lambda a: 1/(1+np.exp(-a))
    n,p = X.shape
    
    # Initialize
    beta = np.zeros(p)
    
    # Update (equiv to IRLS)
    for k in range(L):
        beta_old = beta
        f = X.dot(beta_old)
        w = sigmoid(f) * sigmoid(-f)
        z = f + y / sigmoid(y*f)
        beta = solveWRR(z, X, w, 2*lam)
        if np.sum((beta-beta_old)**2) < eps:
            break
        
    return (beta)

In [287]:
beta0 = solveLRR(y,Xbis,0.002)
beta1 = solveLRR(y2,Xter,0.002)
beta3 = solveLRR(y3,X3,0.002)


pred = np.log(1+np.exp(Xbis.dot(beta0)))
pred2 = np.log(1+np.exp(Xter.dot(beta1)))
pred3 = np.log(1+np.exp(X3.dot(beta3)))

In [288]:
def classificate(data):
    data[data >= 0.5] = 1
    data[data < 0.5] = -1
    return(data)


pred = classificate(pred)
pred2 = classificate(pred2)
pred3 = classificate(pred3)

In [289]:
def error(ypred, ytrue):
    e = (ypred != ytrue).mean()
    return (e)

error(pred,y)

0.282

In [290]:
error(pred2,y2)

0.354

In [291]:
error(pred3,y3)

0.243

## Prediction des Test sets

In [132]:
predT1 = np.log(1+np.exp(Xtest1.dot(beta0)))
predT2 = np.log(1+np.exp(Xtest2.dot(beta1)))
predT3 = np.log(1+np.exp(Xtest3.dot(beta3)))

predT1 = classificate(predT1)
predT2 = classificate(predT2)
predT3 = classificate(predT3)

In [133]:
def remise0(data):
    data[data == -1] = 0
    return(data)

In [134]:
predT1 = remise0(predT1)
predT2 = remise0(predT2)
predT3 = remise0(predT3)

In [149]:
rendu = np.hstack((predT1, predT2,predT3))
data = { "Id": range(3000), "Bound" : rendu}

df = pd.DataFrame(data, columns = ["Id","Bound"])

In [150]:
df = df.astype(int)

Id       int64
Bound    int64
dtype: object

In [151]:
df.to_csv("/users/dorian/Documents/M2_MASH/Kernel Methods/Data/resultats.csv", index = False)

## Essai en comptant le nombre d 'occurences

In [328]:
def compte(ADN):
    compteurA = sum(elem == "A" for elem in ADN)
    compteurC = sum(elem == "C" for elem in ADN)
    compteurG =sum( elem == "G" for elem in ADN)
    compteurT =sum( elem == "T" for elem in ADN)
    
    return((compteurA,compteurC,compteurG,compteurT))

In [332]:
matrice = np.zeros((n,4))
matrice2 = np.zeros((n,4))
matrice3 = np.zeros((n,4))

for j in range(n):
    matrice[j,] = compte(training.iloc[j,1])
    matrice2[j,] = compte(training2.iloc[j,1])
    matrice3[j,] = compte(training3.iloc[j,1])

In [356]:
weights = solveLRR(y,matrice,0.1)
weights2 = solveLRR(y2,matrice2,0.01)
weights3 = solveLRR(y3,matrice3,0.01)

probas = np.log(1+np.exp(matrice.dot(weights)))
probas2 = np.log(1+np.exp(matrice2.dot(weights2)))
probas3 = np.log(1+np.exp(matrice3.dot(weights3)))

probas = classificate(probas)
probas2 = classificate(probas2)
probas3 = classificate(probas3)

In [358]:
error(probas,y)


0.519

In [359]:
error(probas2,y2)

0.4995

In [360]:
error(probas3,y3)

0.5015

## Kernel logistic regression 

In [375]:
n = Xbis.shape[0]
K1 = np.zeros((n,n))
alpha = 0.01
for i in range(n):
    for j in range(n):
        K1[i,j] = gaussK(Xbis[i,],Xbis[j,],alpha)

In [462]:
n = Xbis.shape[0]
K2 = np.zeros((n,n))
K3 = np.zeros((n,n))
alpha = 0.01
for i in range(n):
    for j in range(n):
        K2[i,j] = gaussK(Xter[i,],Xter[j,],alpha)
        K3[i,j] = gaussK(X3[i,],X3[j,],alpha)

In [548]:
def logistic_loss(u):
    return(np.log(1+np.exp(-u)))
def derive(u):
    return(-logistic_loss(-u))
def derive2(u):
    return(logistic_loss(u)*logistic_loss(-u))


def P(alpha,K,y):
    P = np.eye(K.shape[0])
    product = K.dot(alpha)
    for i in range(K.shape[0]):
        P[i,i] = derive(y[i]*product[i])
    return (P)

def KLL(K,y,lam):
    alpha = np.zeros((K.shape[0],1))
    n_iter = 100
    step = 0.001
    n = K.shape[0]
    P = np.eye(n)

    for i in range(n_iter):
        alpha = alpha - step * ((1/n*K.dot(P(alpha,K,y)).dot(y)).reshape(-1,1)) #+ lam*K.dot(alpha))
    return(alpha)
               
    
    

In [578]:
alpha = np.zeros((K1.shape[0],1))
rd = (K1.dot(P(alpha,K1,y)).dot(y)).reshape(-1,1)
print(alpha.shape,rd.shape, (K1.dot(alpha)).shape)


for i in range(30):
    alpha = alpha - 0.01 * ((1/n*K1.dot(P(alpha,K1,y)).dot(y)).reshape(-1,1) + 0.1*K1.dot(alpha))
print(alpha)

(2000, 1) (2000, 1) (2000, 1)
[[-0.00555677]
 [-0.00539191]
 [-0.0057093 ]
 ...
 [-0.00544878]
 [-0.00666282]
 [-0.00558923]]


In [576]:
alpha1 = KLL(K1,y,0.01) # Etrange marche pas dans la boucle de KLL mais marche au dessus

TypeError: 'numpy.ndarray' object is not callable

## KLR avec approximation

In [473]:
yprediction = Ridge(K1,y,0.001)
for i in range(yprediction.shape[0]):
    if yprediction[i] >= 0 : 
        yprediction[i] = 1
    else:
        yprediction[i] = -1
        
yprediction2 = Ridge(K2,y2,0.001)
for i in range(yprediction.shape[0]):
    if yprediction2[i] >= 0 :
        yprediction2[i] = 1
    else:
        yprediction2[i] = -1
        
yprediction3 = Ridge(K3,y3,0.001)
for i in range(yprediction.shape[0]):
    if yprediction3[i] >= 0 :
        yprediction3[i] = 1
    else:
        yprediction3[i] = -1

In [475]:
error(yprediction,y)

0.072

In [718]:
error(yprediction2,y2)

0.044

In [506]:
error(yprediction3,y3)

0.076

## Essai avec les test sets


In [540]:
size = Xtest1.shape[0]

In [489]:
alpha1 = get_alpha(K1,y,0.001)
alpha2 = get_alpha(K2,y2,0.001)
alpha3 = get_alpha(K3,y3,0.001)

In [527]:
ytest1 = np.zeros((size,1))
ytest2 = np.zeros((size,1))
ytest3 = np.zeros((size,1))

for i in range(size):
    summ1=0
    summ2 =0
    summ3=0
    for j in range(n):
        summ1 += alpha1[j]*gaussK(Xtest1[i,],Xbis[j,],0.01)
        summ2 += alpha2[j]*gaussK(Xtest2[i,],Xter[j,],0.01)
        summ3 += alpha3[j]*gaussK(Xtest3[i,],X3[j,],0.01)
    ytest1[i] = summ1
    ytest2[i] = summ2
    ytest3[i] = summ3

In [533]:
YT = np.vstack((ytest1,ytest2,ytest3))
for i in range(YT.shape[0]):
    if np.abs(YT[i] - 1) < np.abs(YT[i] + 1):
        YT[i] = 1
    else:
        YT[i] = -1
        
YT = remise0(YT)

In [534]:
data = { "Id": range(3000), "Bound" : YT[:,0]}

df = pd.DataFrame(data, columns = ["Id","Bound"])
df = df.astype(int)

Id       int64
Bound    int64
dtype: object

In [538]:
df.to_csv("/users/dorian/Documents/M2_MASH/Kernel Methods/Data/resultats2.csv", index = False)

## Substring indexation

In [685]:
length_substring = 3
substring = set()
reference = training.iloc[1,1] # Pris au hasard
for i in range((101-length_substring)): #101 longeur de l'ADN
    substring.add(reference[i:i+length_substring])
    
nb_variables = len(substring)

In [686]:
substring = list(substring) # Set non indexable et pas accessible -> conversion list

In [687]:
def create_features(X,nb_var, substring): #nb_var = len(substring)
    n = X.shape[0]
    output = np.zeros((n,nb_var))
    for i in range(n):
        for j in range(len(substring)):
            output[i,j] = X[i].count(substring[j])
    return(output)

def create_GaussKernel(X,sig):
    n = X.shape[0]
    K = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            K[i,j] = gaussK(X[i,],X[j,],sig)  #fonction au debut
    return(K)
    

In [688]:
obs1 = create_features(training.iloc[:,1], nb_variables,substring)
obs2 = create_features(training2.iloc[:,1], nb_variables,substring)
obs3 = create_features(training3.iloc[:,1], nb_variables,substring)

In [639]:
K1 = create_GaussKernel(obs1,0.01)

In [640]:
K2 = create_GaussKernel(obs2,0.01)
K3 = create_GaussKernel(obs3,0.01)

## Erreur training

In [713]:
alpha1 = get_alpha(K1,y,0.001) # A fazire avec 0.01 eet 0.1
alpha2 = get_alpha(K2,y2,0.001)
alpha3 = get_alpha(K3,y3,0.001)

y_tr1 = Ridge(K1,y,0.01)
y_tr2 = Ridge(K2,y2,0.01)
y_tr3 = Ridge(K3,y3,0.01)

In [694]:
y_tr1[y_tr1 > 0] = 1
y_tr1[y_tr1 <= 0] = -1
error(y_tr1,y)

0.3265

In [695]:
y_tr2[y_tr2 > 0] = 1
y_tr2[y_tr2 <= 0] = -1
error(y_tr2,y2)

0.3245

In [714]:
ytest1 = np.zeros((size,1))
ytest2 = np.zeros((size,1))
ytest3 = np.zeros((size,1))

for i in range(size):
    summ1=0
    summ2 =0
    summ3=0
    for j in range(n):
        summ1 += alpha1[j]*gaussK(Xtest1[i,],Xbis[j,],0.01)
        summ2 += alpha2[j]*gaussK(Xtest2[i,],Xter[j,],0.01)
        summ3 += alpha3[j]*gaussK(Xtest3[i,],X3[j,],0.01)
    ytest1[i] = summ1
    ytest2[i] = summ2
    ytest3[i] = summ3

In [715]:
YT = np.vstack((ytest1,ytest2,ytest3))
for i in range(YT.shape[0]):
    if np.abs(YT[i] - 1) < np.abs(YT[i] + 1):
        YT[i] = 1
    else:
        YT[i] = -1
        
YT = remise0(YT)
data = { "Id": range(3000), "Bound" : YT[:,0]}

df = pd.DataFrame(data, columns = ["Id","Bound"])
df = df.astype(int)

Id       int64
Bound    int64
dtype: object

In [717]:
df.to_csv("/users/dorian/Documents/M2_MASH/Kernel Methods/Data/resultats3.csv", index = False)

# SVM

In [ ]:
def gaussian_kernel(x, y, alpha=5):
    return np.exp(-alpha*np.linalg.norm(x-y)**2)

def precision(x, y):
    rep = (np.array(x).flatten()*np.array(y).flatten() > 0)*1
    return np.mean(rep)

def SVM_hyperparameters(alpha, Xtr, Ytr, kernel, n_train, C):
    
    n = Xtr.shape[0]
    n_valid = n - n_train
    
    X_train = Xtr[0:n_train]
    X_valid = Xtr[n_train:n]
    
    y = Ytr*2 - 1

    y_train = y[0:n_train]
    y_valid = y[n_train:n]
    
    Y_diag = np.diag(y_train) * 1.
    
    y_train = y_train.reshape(-1,1) * 1.
    
    K = np.zeros((n_train, n_train))
    for i in range(n_train):
        for j in range(n_train):
            K[i,j] = kernel(X_train[i], X_train[j], alpha)
    
    borne_sup = np.repeat(C, n_train)
    borne_inf = np.zeros(n_train)
    borne = np.concatenate((borne_sup, borne_inf), axis=None)
    borne = borne.reshape(-1, 1) * 1.
    
    ing = np.concatenate((Y_diag, -Y_diag), axis=0)
        
    P = cvxopt_matrix(K, tc='d')
    q = cvxopt_matrix(-y_train, tc='d')
    h = cvxopt_matrix(borne, tc='d')
    G = cvxopt_matrix(ing, tc='d')
    
    sol = cvxopt_solvers.qp(P,q,G,h)
    a = np.array(sol['x']).flatten()
        
    prediction = np.zeros(n_valid)
    for i in range(n_valid):
        pred = 0
        for j in range(n_train):
            pred = pred + a[j]*kernel(X_train[j], X_valid[i], alpha)
        prediction[i] = pred  
        
    return precision(prediction_final, y_valid)

def SVM_predict(alpha, Xtr, Ytr, kernel, n_train, C, biais = False):
    
    n = Xtr.shape[0]
    n_valid = n - n_train
    
    X_train = Xtr[0:n_train]
    X_valid = Xtr[n_train:n]
    
    y = Ytr*2 - 1

    y_train = y[0:n_train]
    
    Y_diag = np.diag(y_train) * 1.
    
    y_train = y_train.reshape(-1,1) * 1.
    
    K = np.zeros((n_train, n_train))
    for i in range(n_train):
        for j in range(n_train):
            K[i,j] = kernel(X_train[i], X_train[j], alpha)
    
    borne_sup = np.repeat(C, n_train)
    borne_inf = np.zeros(n_train)
    borne = np.concatenate((borne_sup, borne_inf), axis=None)
    borne = borne.reshape(-1, 1) * 1.
    
    ing = np.concatenate((Y_diag, -Y_diag), axis=0)
        
    P = cvxopt_matrix(K, tc='d')
    q = cvxopt_matrix(-y_train, tc='d')
    h = cvxopt_matrix(borne, tc='d')
    G = cvxopt_matrix(ing, tc='d')
    
    sol = cvxopt_solvers.qp(P,q,G,h)
    a = np.array(sol['x']).flatten()
        
    prediction = np.zeros(n_valid)
    for i in range(n_valid):
        pred = 0
        for j in range(n_train):
            pred = pred + a[j]*kernel(X_train[j], X_valid[i], alpha)
        prediction[i] = pred  
     
    b = 0
        
    y_train = y_train.flatten().astype(int)
    for i in range(n_train):
        b = b + y_train[i]
        s = 0
        for j in range(n_train):
            s = s + a[j]*kernel(X_train[j], X_train[i], alpha)/(C*n_train)
        b = b - s
    b = b/n_train
    
    if biais:
        prediction_final = np.sign(prediction+b)
    else : 
        prediction_final = np.sign(prediction)
    
    return prediction_final

## Grid search first set

In [ ]:
list_C_1 = np.linspace(0.01, 0.1, 10)
list_s_1 = np.linspace(400, 700, 31) 

result_svm1 = np.zeros((10, 31))
for i in range(len(list_C_1)):
    for j in range(len(list_s_1)):
        result_svm1[i, j] = SVM_hyperparameters(list_s_1[j], Xtr0_mat100, Y0, gaussian_kernel, 1800, list_C_1[i])
        
c1_svm, s1_svm = np.unravel_index(result_svm1.argmax(), result_svm1.shape)

## Grid search second set

In [ ]:
list_C_2 = np.linspace(0.01, 0.1, 10)
list_s_2 = np.linspace(200, 500, 31) 

result_svm2 = np.zeros((10, 31))
for i in range(len(list_C_2)):
    for j in range(len(list_s_2)):
        result_svm2[i, j] = SVM_hyperparameters(list_s_2[j], Xtr1_mat100, Y1, gaussian_kernel, 1800, list_C_2[i])
        
c2_svm, s2_svm = np.unravel_index(result_svm2.argmax(), result_svm2.shape)

## Grid search third set

In [ ]:
list_C_3 = np.linspace(0.01, 0.1, 10)
list_s_3 = np.linspace(50, 100, 101) 

result_svm3 = np.zeros((10, 101))
for i in range(len(list_C_3)):
    for j in range(len(list_s_3)):
        result_svm3[i, j] = SVM_hyperparameters(list_s_3[j], Xtr2_mat100, Y2, gaussian_kernel, 1800, list_C_3[i])
        
c3_svm, s3_svm = np.unravel_index(result_svm3.argmax(), result_svm3.shape)

## Predictions 

In [ ]:
Y_svm1 = SVM_predict(list_s_1[s1_svm], X0, Y0, gaussian_kernel, 2000, list_C_1[c1_svm])
Y_svm2 = SVM_predict(list_s_2[s2_svm], X1, Y1, gaussian_kernel, 2000, list_C_2[c2_svm])
Y_svm3 = SVM_predict(list_s_3[s3_svm], X2, Y2, gaussian_kernel, 2000, list_C_3[c3_svm])

In [ ]:
Y_svm = np.concatenate((Y_svm1, Y_svm2, Y_svm3), axis=None)
Y_svm = (Y_svm+1)/2
Y_svm = Y_svm.astype(int)

In [ ]:
prediction = pd.DataFrame({'Id' : np.arange(3000), "Bound": Y_svm})

In [ ]:
prediction.to_csv('prediction_svm.csv', index=False)